In [2]:
import pandas as pd
import boto3
import json
import datetime

### Getting Tweets

In [3]:
df_tweets = pd.read_csv("BTC_tweets.csv")

In [4]:
df_tweets = df_tweets.drop('Unnamed: 0', axis=1)

In [5]:
df_tweets

,text,timestamp
0,Today's Sophia BTC Mood: Bearish #Crypto #Cryp...,2023-05-20T00:00:00Z
1,"App Development On Bitcoin?, Beeple Bitboy NFT...",2023-05-20T00:00:00Z
2,PREÇO BITCOIN BRASIL (BTCBRL)\nValor: R$ 134.8...,2023-05-20T00:00:00Z
3,"$BTC Price: $26,875 \n#Bitcoin #BTC #BitcoinPr...",2023-05-20T00:00:00Z
4,#Bitcoin #Crypto #Blockchain #Cryptocurrency #...,2023-05-20T01:00:00Z
...,...,...
617,RT @darkducktrader: 🛰 SpaceX will deliver a pr...,2023-05-24T23:00:00Z
618,#Bitcoin #Crypto #Blockchain #Cryptocurrency #...,2023-05-24T23:00:00Z
619,#Bitcoin #Crypto #Blockchain #Cryptocurrency #...,2023-05-24T23:00:00Z
620,#Bitcoin #Crypto #Blockchain #Cryptocurrency #...,2023-05-24T23:00:00Z


#### Running Comprehend

In [6]:
comprehend = boto3.client(service_name='comprehend', region_name='eu-west-1')

In [9]:
df_tweets['sentiment'] = df_tweets['text'].apply(lambda x: json.loads(json.dumps(comprehend.detect_sentiment(Text=x, LanguageCode='en'), sort_keys=True, indent=4))['Sentiment'])

In [10]:
df_tweets

,text,timestamp,sentiment
0,Today's Sophia BTC Mood: Bearish #Crypto #Cryp...,2023-05-20T00:00:00Z,NEUTRAL
1,"App Development On Bitcoin?, Beeple Bitboy NFT...",2023-05-20T00:00:00Z,NEUTRAL
2,PREÇO BITCOIN BRASIL (BTCBRL)\nValor: R$ 134.8...,2023-05-20T00:00:00Z,NEUTRAL
3,"$BTC Price: $26,875 \n#Bitcoin #BTC #BitcoinPr...",2023-05-20T00:00:00Z,NEUTRAL
4,#Bitcoin #Crypto #Blockchain #Cryptocurrency #...,2023-05-20T01:00:00Z,NEUTRAL
...,...,...,...
617,RT @darkducktrader: 🛰 SpaceX will deliver a pr...,2023-05-24T23:00:00Z,NEUTRAL
618,#Bitcoin #Crypto #Blockchain #Cryptocurrency #...,2023-05-24T23:00:00Z,NEUTRAL
619,#Bitcoin #Crypto #Blockchain #Cryptocurrency #...,2023-05-24T23:00:00Z,NEUTRAL
620,#Bitcoin #Crypto #Blockchain #Cryptocurrency #...,2023-05-24T23:00:00Z,NEUTRAL


In [26]:
df_tweets['year'] = df_tweets['timestamp'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ").year)
df_tweets['month'] = df_tweets['timestamp'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ").month)
df_tweets['day'] = df_tweets['timestamp'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ").day)
df_tweets['day_of_week'] = df_tweets['timestamp'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ").weekday())


In [38]:
cols = list(df_tweets.columns)[3:] + list(df_tweets.columns)[:3]

In [40]:
cols

['year', 'month', 'day', 'day_of_week', 'text', 'timestamp', 'sentiment']

In [41]:
df_tweets = df_tweets[cols]

In [44]:
df_tweets.to_csv('BTC_tweets_cleaned.csv', index=False) 

### Aggregating for Regression

In [56]:
df_tweets_agg = pd.DataFrame(df_tweets.groupby('timestamp')['sentiment'].agg(pd.Series.mode)).reset_index()

In [59]:
df_tweets_agg = df_tweets.rename(columns = {'timestamp': 'open_time'})

In [69]:
df_tweets_agg.head()

,open_time,sentiment
0,2023-05-20T00:00:00Z,NEUTRAL
1,2023-05-20T01:00:00Z,NEUTRAL
2,2023-05-20T02:00:00Z,NEUTRAL
3,2023-05-20T03:00:00Z,NEUTRAL
4,2023-05-20T04:00:00Z,NEUTRAL


#### Importing Bitcoin price data

In [63]:
df_btc = pd.read_csv("BTC_USDT_20230519_20230525_128.csv")

In [64]:
df_btc.head()

,open_time,close_time,open,high,low,close,volume,quote_asset_volume,num_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,open_timestamp,close_timestamp
0,"05/24/2023, 23:00:00","05/24/2023, 23:59:59",26348.52,26394.16,26318.25,26329.01,712.51394,1.877893e+07,16479,376.83127,9.932271e+06,0,1684969200000,1684972799999
1,"05/24/2023, 22:00:00","05/24/2023, 22:59:59",26368.66,26406.00,26313.47,26348.52,746.24121,1.967040e+07,19907,325.98884,8.592227e+06,0,1684965600000,1684969199999
2,"05/24/2023, 21:00:00","05/24/2023, 21:59:59",26404.18,26464.65,26310.48,26368.66,1223.45586,3.229833e+07,27225,642.68037,1.696504e+07,0,1684962000000,1684965599999
3,"05/24/2023, 20:00:00","05/24/2023, 20:59:59",26252.34,26486.46,26242.62,26404.18,2878.77545,7.593704e+07,47336,1539.53932,4.060586e+07,0,1684958400000,1684961999999
4,"05/24/2023, 19:00:00","05/24/2023, 19:59:59",26220.55,26397.64,26201.67,26252.33,2349.96235,6.179576e+07,40509,1171.28517,3.080221e+07,0,1684954800000,1684958399999


In [65]:
df_btc['open_time'] = df_btc['open_time'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y, %H:%M:%S").isoformat() +'Z')

In [72]:
df_btc.join(df_tweets.set_index('open_time'), on = "open_time", how = "inner")


,open_time,close_time,open,high,low,close,volume,quote_asset_volume,num_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,open_timestamp,close_timestamp,sentiment
0,2023-05-24T23:00:00Z,"05/24/2023, 23:59:59",26348.52,26394.16,26318.25,26329.01,712.51394,1.877893e+07,16479,376.83127,9.932271e+06,0,1684969200000,1684972799999,NEUTRAL
1,2023-05-24T22:00:00Z,"05/24/2023, 22:59:59",26368.66,26406.00,26313.47,26348.52,746.24121,1.967040e+07,19907,325.98884,8.592227e+06,0,1684965600000,1684969199999,NEUTRAL
2,2023-05-24T21:00:00Z,"05/24/2023, 21:59:59",26404.18,26464.65,26310.48,26368.66,1223.45586,3.229833e+07,27225,642.68037,1.696504e+07,0,1684962000000,1684965599999,NEUTRAL
3,2023-05-24T20:00:00Z,"05/24/2023, 20:59:59",26252.34,26486.46,26242.62,26404.18,2878.77545,7.593704e+07,47336,1539.53932,4.060586e+07,0,1684958400000,1684961999999,NEUTRAL
4,2023-05-24T19:00:00Z,"05/24/2023, 19:59:59",26220.55,26397.64,26201.67,26252.33,2349.96235,6.179576e+07,40509,1171.28517,3.080221e+07,0,1684954800000,1684958399999,NEUTRAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2023-05-20T04:00:00Z,"05/20/2023, 04:59:59",26865.59,26873.63,26851.11,26873.63,337.53628,9.066916e+06,15527,147.76932,3.969283e+06,0,1684555200000,1684558799999,NEUTRAL
116,2023-05-20T03:00:00Z,"05/20/2023, 03:59:59",26839.63,26874.74,26839.62,26865.58,318.51343,8.554464e+06,16334,171.26475,4.599587e+06,0,1684551600000,1684555199999,NEUTRAL
117,2023-05-20T02:00:00Z,"05/20/2023, 02:59:59",26858.98,26870.05,26832.36,26839.63,327.99884,8.807418e+06,12311,138.35650,3.715217e+06,0,1684548000000,1684551599999,NEUTRAL
118,2023-05-20T01:00:00Z,"05/20/2023, 01:59:59",26845.71,26885.86,26845.54,26858.98,270.63860,7.269188e+06,14041,137.13224,3.683199e+06,0,1684544400000,1684547999999,NEUTRAL
